In [1]:
# 预训练代码


from transformers import AutoTokenizer, AutoModelForCausalLM
from model.configuration_qwen_agentic import CompressorConfig, ConverterConfig, DecoderConfig
from model.modeling_qwen_agentic import Qwen3AgenticConfig, Qwen3AgenticModel, Qwen3Agentic

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
config = Qwen3AgenticConfig.from_pretrained("./model")
config

/home/slgx/miniconda3/envs/memagent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Qwen3AgenticConfig {
  "architectures": [
    "Qwen3Agentic"
  ],
  "auto_map": {
    "AutoConfig": "configuration_qwen_agentic.Qwen3AgenticConfig",
    "AutoModel": "modeling_qwen_agentic.Qwen3Agentic",
    "AutoModelForCausalLM": "modeling_qwen_agentic.Qwen3Agentic"
  },
  "compressor_config": {
    "architectures": [
      "Qwen3ForCausalLM"
    ],
    "attention_bias": false,
    "attention_dropout": 0.0,
    "base_model": "/home/slgx/yuhong/base_model/Qwen/Qwen3-0.6B",
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 1024,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_types": [
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention

In [2]:

if hasattr(config.compressor_config, 'base_model'):
    print(config.compressor_config.base_model)


/home/slgx/yuhong/base_model/Qwen/Qwen3-0.6B


In [2]:
model = Qwen3Agentic.from_pretrained("/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic", device_map="auto").to("cuda")
model

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.20s/it]


Qwen3Agentic(
  (compressor): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qw

In [3]:
print(model.tokenizer)

AttributeError: 'Qwen3Agentic' object has no attribute 'tokenizer'

In [ ]:
model.tokenizer.save_pretrained("/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic")

('/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/tokenizer_config.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/special_tokens_map.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/chat_template.jinja',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/vocab.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/merges.txt',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/added_tokens.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/tokenizer.json')

In [5]:
model.config

Qwen3AgenticConfig {
  "architectures": [
    "Qwen3Agentic"
  ],
  "auto_map": {
    "AutoConfig": "configuration_qwen_agentic.Qwen3AgenticConfig",
    "AutoModel": "modeling_qwen_agentic.Qwen3Agentic",
    "AutoModelForCausalLM": "modeling_qwen_agentic.Qwen3Agentic"
  },
  "compressor_config": {
    "architectures": [
      "Qwen3ForCausalLM"
    ],
    "attention_bias": false,
    "attention_dropout": 0.0,
    "base_model": "/home/slgx/yuhong/base_model/Qwen/Qwen3-0.6B",
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 1024,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_types": [
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention

In [7]:
model.compressor

Qwen3Model(
  (embed_tokens): Embedding(151936, 1024)
  (layers): ModuleList(
    (0-27): 28 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
    )
  )
  (norm): Qwen3RMSNorm((102

In [ ]:
print(model)

Qwen3Agentic(
  (compressor): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qw

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

In [12]:
prompt = "Hello, what's result of 100*20+2?"
message = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
text

"<|im_start|>user\nHello, what's result of 100*20+2?<|im_end|>\n<|im_start|>assistant\n"

In [13]:
model_input = tokenizer([text], return_tensors="pt").to("cuda")
generate_ids = model.decoder.generate(**model_input, max_new_tokens=1024)

In [14]:
output_ids = generate_ids[0][len(model_input.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, let's see. The user is asking for the result of 100 multiplied by 20 plus 2. Alright, so first, I need to remember the order of operations here. In math problems, multiplication comes before addition unless there are parentheses that change the order. So, in this case, it's 100 times 20 first, and then add 2 to that result.

Let me break it down step by step. First, calculate 100 multiplied by 20. Hmm, 100 times 20. Well, 100 times 10 is 1000, so 100 times 20 would be double that, right? So 1000 times 2 is 2000. So that part is 2000.

Now, add 2 to that. So 2000 plus 2. That seems straightforward. 2000 plus 2 is 2002. Let me just verify that again. 100*20 is definitely 2000, and adding 2 gives 2002. I don't think I made any mistakes here. Maybe I can check using another method. For example, 100*20 is the same as 20*100, which is 2000. Then adding 2, so yes, 2002. 

Wait, is there any chance the user might have meant something else, like 100*(20+2)? But n

In [17]:
model.converter

MemoryConverter(
  (gate_proj): Linear(in_features=3072, out_features=3072, bias=False)
  (up_proj): Linear(in_features=3072, out_features=3072, bias=False)
  (down_proj): Linear(in_features=3072, out_features=3072, bias=False)
  (act_fn): SiLU()
  (rms_norm): RMSNorm((3072,), eps=1e-06, elementwise_affine=True)
)

In [20]:
!pip install huggingface_hub -q
# huggingface-cli login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from huggingface_hub import HfApi, create_repo
api = HfApi()

In [ ]:
total_params = 0
trainable_params = 0

for param in model.converter.parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel() 

print(f"Converter 总参数量: {total_params:,}")
print(f"Converter 可训练参数量: {trainable_params:,}")
print(f"Converter 参数量 (M): {total_params / 1e6:.2f}M")


Converter 总参数量: 28,314,624
Converter 可训练参数量: 28,314,624
Converter 参数量 (M): 28.31M


In [7]:
model.save_pretrained("/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic")

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

/home/slgx/miniconda3/envs/memagent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
tokenizer.save_pretrained("/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic")

('/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/tokenizer_config.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/special_tokens_map.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/chat_template.jinja',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/vocab.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/merges.txt',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/added_tokens.json',
 '/home/slgx/yuhong/base_model/Qwen/Qwen3-Agentic/tokenizer.json')

In [1]:
model

NameError: name 'model' is not defined